In [1]:
from datasets import load_dataset
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, GenerationConfig, TrainingArguments, Trainer
import torch
import time
import evaluate
import pandas as pd
import numpy as np

d:\llmenv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
huggingface_dataset_name = "knkarthick/dialogsum"
dataset = load_dataset(huggingface_dataset_name)
dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'dialogue', 'summary', 'topic'],
        num_rows: 12460
    })
    validation: Dataset({
        features: ['id', 'dialogue', 'summary', 'topic'],
        num_rows: 500
    })
    test: Dataset({
        features: ['id', 'dialogue', 'summary', 'topic'],
        num_rows: 1500
    })
})

In [3]:
dataset['train'][0]

{'id': 'train_0',
 'dialogue': "#Person1#: Hi, Mr. Smith. I'm Doctor Hawkins. Why are you here today?\n#Person2#: I found it would be a good idea to get a check-up.\n#Person1#: Yes, well, you haven't had one for 5 years. You should have one every year.\n#Person2#: I know. I figure as long as there is nothing wrong, why go see the doctor?\n#Person1#: Well, the best way to avoid serious illnesses is to find out about them early. So try to come at least once a year for your own good.\n#Person2#: Ok.\n#Person1#: Let me see here. Your eyes and ears look fine. Take a deep breath, please. Do you smoke, Mr. Smith?\n#Person2#: Yes.\n#Person1#: Smoking is the leading cause of lung cancer and heart disease, you know. You really should quit.\n#Person2#: I've tried hundreds of times, but I just can't seem to kick the habit.\n#Person1#: Well, we have classes and some medications that might help. I'll give you more information before you leave.\n#Person2#: Ok, thanks doctor.",
 'summary': "Mr. Smith'

In [4]:
example = dataset['train'][0]
dialogure_lines = example['dialogue'].split('\n')

for line in dialogure_lines:
    print(line)

#Person1#: Hi, Mr. Smith. I'm Doctor Hawkins. Why are you here today?
#Person2#: I found it would be a good idea to get a check-up.
#Person1#: Yes, well, you haven't had one for 5 years. You should have one every year.
#Person2#: I know. I figure as long as there is nothing wrong, why go see the doctor?
#Person1#: Well, the best way to avoid serious illnesses is to find out about them early. So try to come at least once a year for your own good.
#Person2#: Ok.
#Person1#: Let me see here. Your eyes and ears look fine. Take a deep breath, please. Do you smoke, Mr. Smith?
#Person2#: Yes.
#Person1#: Smoking is the leading cause of lung cancer and heart disease, you know. You really should quit.
#Person2#: I've tried hundreds of times, but I just can't seem to kick the habit.
#Person1#: Well, we have classes and some medications that might help. I'll give you more information before you leave.
#Person2#: Ok, thanks doctor.


In [5]:
# Load the Model
model_name = 'google/flan-t5-base'

original_model = AutoModelForSeq2SeqLM.from_pretrained(model_name , torch_dtype = torch.bfloat16)
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [6]:
#Example :

prompt = "Hello world"
tokens = tokenizer(prompt, return_tensors="pt") #pt is for pytorch

print(tokens["input_ids"])
print(tokenizer.decode(tokens["input_ids"][0],skip_special_tokens=True))

tensor([[8774,  296,    1]])
Hello world


In [7]:
# To check the trainable parameters
def print_number_of_trainable_model_parameters(model):
    trainable_model_params = 0
    all_model_params = 0

    for _, param in model.named_parameters():
        all_model_params += param.numel()
        if param.requires_grad:
            trainable_model_params += param.numel()
    return f"trainable model parameters: {trainable_model_params}\n All model parameters:{all_model_params} \n Percentage of trainable model parameters: {100 * trainable_model_params / all_model_params:.2f}%"
print(print_number_of_trainable_model_parameters(original_model))

trainable model parameters: 247577856
 All model parameters:247577856 
 Percentage of trainable model parameters: 100.00%


# Test the Model with Zero Shot Inferencing

In [8]:
# Lets utilize this dialogue for inference
example = dataset['test'][200]
dialogure_lines = example['dialogue'].split('\n')

for line in dialogure_lines:
    print(line)

print(f"Summary of the Dialogure is: {dataset['test'][200]['summary']}")

#Person1#: Have you considered upgrading your system?
#Person2#: Yes, but I'm not sure what exactly I would need.
#Person1#: You could consider adding a painting program to your software. It would allow you to make up your own flyers and banners for advertising.
#Person2#: That would be a definite bonus.
#Person1#: You might also want to upgrade your hardware because it is pretty outdated now.
#Person2#: How can we do that?
#Person1#: You'd probably need a faster processor, to begin with. And you also need a more powerful hard disc, more memory and a faster modem. Do you have a CD-ROM drive?
#Person2#: No.
#Person1#: Then you might want to add a CD-ROM drive too, because most new software programs are coming out on Cds.
#Person2#: That sounds great. Thanks.
Summary of the Dialogure is: #Person1# teaches #Person2# how to upgrade software and hardware in #Person2#'s system.


In [9]:
index = 200
dialogue = dataset['test'][index]['dialogue']
summary = dataset['test'][index]['summary']

prompt = f""""
Summarize the following conversation.
{dialogue}

Summary:
"""

inputs = tokenizer(prompt, return_tensors='pt')
output = tokenizer.decode(
    original_model.generate(inputs["input_ids"],
    max_new_tokens = 200,
    )[0],
    skip_special_tokens=True
)
dash_line = '-'.join('' for x in range(80))
print(dash_line)

print(f'Input Prompt: \n{prompt}')
print(dash_line)

print(f"BaseLine Human Summary: \n{summary}\n")
print(dash_line)

print(f"Model Generated - Zero Shot :\n{output}")

-------------------------------------------------------------------------------
Input Prompt: 
"
Summarize the following conversation.
#Person1#: Have you considered upgrading your system?
#Person2#: Yes, but I'm not sure what exactly I would need.
#Person1#: You could consider adding a painting program to your software. It would allow you to make up your own flyers and banners for advertising.
#Person2#: That would be a definite bonus.
#Person1#: You might also want to upgrade your hardware because it is pretty outdated now.
#Person2#: How can we do that?
#Person1#: You'd probably need a faster processor, to begin with. And you also need a more powerful hard disc, more memory and a faster modem. Do you have a CD-ROM drive?
#Person2#: No.
#Person1#: Then you might want to add a CD-ROM drive too, because most new software programs are coming out on Cds.
#Person2#: That sounds great. Thanks.

Summary:

-------------------------------------------------------------------------------
BaseLi

# Performance Full Fine Tuning

In [10]:
def tokenize_function(example):
    start_prompt = "Summarize the following conversation. \n\n"
    end_prompt = "\n\n Summary:"

    prompt = [start_prompt + dialogue + end_prompt for dialogue in example["dialogue"]]
    example['input_ids'] = tokenizer(prompt, truncation = True, padding = "max_length", return_tensors='pt').input_ids
    example['labels'] = tokenizer(example['summary'], truncation = True, padding='max_length', return_tensors='pt').input_ids

    return example

# The dataset actually contains 3 diff splits: train, validation, test.ipynb
# The tokenize_function code is handling all data across all splits in batches.

tokenized_datasets = dataset.map(tokenize_function, batched=True)
tokenized_datasets = tokenized_datasets.remove_columns(['id','topic','dialogue','summary',])

In [11]:
# Now lets check the shapes of all three parts of the dataset but we will first take
# a subset to the data

tokenized_datasets = tokenized_datasets.filter(lambda example, index: index %100 ==0, with_indices=True)
print(f"Shapes of the datasets:")

print(f"Training: {tokenized_datasets['train'].shape}")
print(f"Validation: {tokenized_datasets['validation'].shape}")
print(f"Test: {tokenized_datasets['test'].shape}")

print(tokenized_datasets)

Filter: 100%|██████████| 1500/1500 [00:00<00:00, 3229.77 examples/s]

Shapes of the datasets:
Training: (125, 2)
Validation: (5, 2)
Test: (15, 2)
DatasetDict({
    train: Dataset({
        features: ['input_ids', 'labels'],
        num_rows: 125
    })
    validation: Dataset({
        features: ['input_ids', 'labels'],
        num_rows: 5
    })
    test: Dataset({
        features: ['input_ids', 'labels'],
        num_rows: 15
    })
})


# Fine Tune the Model with PreProcessed Dataset

In [ ]:
# Now we will utilize the builtin Hugging Face Trainer Class

output_dir = f'./dialogue-summary-training-{str(int(time.time()))}'

training_args = TrainingArguments(
    output_dir = output_dir,
    learning_rate=1e-5,
    num_train_epochs=1,
    weight_decay=0.01,
    logging_steps=1,
    max_steps=1,
    per_device_train_batch_size=2, # added for gpu but didnt workout
    per_device_eval_batch_size=2, #added for gpu but didnt workout
    fp16=True, #added for gpu but didnt workout
)

#device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

trainer = Trainer (
    model = original_model,
    args = training_args,
    train_dataset=tokenized_datasets['train'],
    eval_dataset = tokenized_datasets['validation']
)


max_steps is given, it will override any value given in num_train_epochs


In [20]:
trainer.train()

Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


Step,Training Loss
1,46.500000


TrainOutput(global_step=1, training_loss=46.5, metrics={'train_runtime': 521.9627, 'train_samples_per_second': 0.004, 'train_steps_per_second': 0.002, 'total_flos': 1369514704896.0, 'train_loss': 46.5, 'epoch': 0.015873015873015872})

In [21]:
# Save the model
trained_model_dir = "./trained_model"
trainer.save_model(trained_model_dir)



In [22]:
#load the saved model
trained_model = AutoModelForSeq2SeqLM.from_pretrained(trained_model_dir)

# Evaluation

In [25]:
# Tokenize the prompt
input_ids = tokenizer(prompt, return_tensors='pt').input_ids


#Ensure that input_ids are on the same device and the model.
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
input_ids = input_ids.to(device)
original_model.to(device)
trained_model.to(device)


#Generate outputs using the original model before training
generation_config = GenerationConfig(max_new_tokens=200,num_beams=1)
original_model_outputs = original_model.generate(input_ids=input_ids, generation_config=generation_config)
original_model_text_output = tokenizer.decode(original_model_outputs[0],skip_special_tokens=True)

#Generate outputs using the trained model after training
trained_model_outputs = trained_model.generate(input_ids = input_ids, generation_config=generation_config)
trained_model_text_output = tokenizer.decode(trained_model_outputs[0],skip_special_tokens=True)


human_baseline_summary = summary
dash_line = '-' *50

print(dash_line)
print(f"Baseline Human Summary: \n{human_baseline_summary}")
print(dash_line)
print(f"Original Model : \n {original_model_text_output}")
print(dash_line)
print(f"Trained Model: \n {trained_model_text_output}")




--------------------------------------------------
Baseline Human Summary: 
#Person1# teaches #Person2# how to upgrade software and hardware in #Person2#'s system.
--------------------------------------------------
Original Model : 
 #Person1#: I'm thinking of upgrading your computer hardware. #Person2#: I'm not sure what I'm looking at. #Person1#: I'm not sure what I need. #Person2#: I'm not sure what I need. #Person1#: I'm not sure what I need. #Person2#: I'm not sure what I need. #Person1#: I'm not sure what I need. #Person2#: I'm not sure what I need. #Person1#: I'm not sure what I need. #Person2#: I'm not sure what I need. #Person1#: I'm not sure what I need. #Person2#: I'm not sure what I need. #Person1#: I'm not sure.
--------------------------------------------------
Trained Model: 
 #Person1#: I'm thinking of upgrading my computer.


# Evaluate the Model Quantitatively with ROUGE Metric

In [26]:
rouge = evaluate.load('rouge')

In [28]:
dialogue = dataset['test'][0:10]['dialogue']
human_baseline_summaries = dataset['test'][0:10]['summary']

original_model_summaries = []
instruct_model_summaries = []

for _, dialogure in enumerate(dialogue):
    prompt = f""""Summarize the following conversation
    {dialogure}
    Summary:

    """
    input_ids = tokenizer(prompt, return_tensors='pt').input_ids
    #Ensure the the input_ids and mode are on the same device
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    input_ids = input_ids.to(device)

    original_model_outputs = original_model.generate(input_ids=input_ids,generation_config = GenerationConfig(max_new_tokens=200,num_beams=1))
    original_model_text_output = tokenizer.decode(original_model_outputs[0],skip_special_tokens=True)
    original_model_summaries.append(original_model_text_output)

    instruct_model_outputs = trained_model.generate(input_ids = input_ids,generation_config = GenerationConfig(max_new_tokens=200, num_beams=1))
    instruct_model_text_output = tokenizer.decode(instruct_model_outputs[0],skip_special_tokens=True)
    instruct_model_summaries.append(instruct_model_text_output)

zipped_summaries = list(zip(human_baseline_summaries,original_model_summaries,instruct_model_summaries))
df = pd.DataFrame(zipped_summaries,columns=['Human_baseline_Summaries','Original_model_summaries','instruct_model_summaries'])
df

,Human_baseline_Summaries,Original_model_summaries,instruct_model_summaries
0,Ms. Dawson helps #Person1# to write a memo to ...,"""",The following memo is to be distributed to all...
1,In order to prevent employees from wasting tim...,Employees are required to sign a dictation bef...,The following memo is to be distributed to all...
2,Ms. Dawson takes a dictation for #Person1# abo...,Employees are required to take a dictation fro...,The following memo is to be distributed to all...
3,#Person2# arrives late because of traffic jam....,The conversation is about the traffic james in...,The traffic jam at the Carrefour intersection ...
4,#Person2# decides to follow #Person1#'s sugges...,The person who is driving to work is upset abo...,The traffic jam at the Carrefour intersection ...
5,#Person2# complains to #Person1# about the tra...,#Person1: You're finally here. What took so lo...,The traffic jam at the Carrefour intersection ...
6,#Person1# tells Kate that Masha and Hero get d...,#Person1: Masha and Hero are getting divorced.,Masha and Hero are getting divorced.
7,#Person1# tells Kate that Masha and Hero are g...,Masha and Hero are getting divorced.,Masha and Hero are getting divorced.
8,#Person1# and Kate talk about the divorce betw...,Masha and Hero are getting divorced.,Masha and Hero are getting divorced.
9,#Person1# and Brian are at the birthday party ...,"#Person1: Thank you, Brian. #Person2: I hope y...","#Person1: Happy Birthday, Brian. #Person2: Tha..."


In [36]:
from rouge import Rouge
rouge = Rouge()


original_model_results = rouge.get_scores(
    original_model_summaries,
    human_baseline_summaries[0:len(original_model_summaries)],
    )

instruct_model_results = rouge.get_scores(
    instruct_model_summaries,
    human_baseline_summaries[0:len(instruct_model_summaries)],
    )

print('Original Model:')
print(original_model_results)

print('Instruct Model:')
print(instruct_model_results)

Original Model:
[{'rouge-1': {'r': 0.0, 'p': 0.0, 'f': 0.0}, 'rouge-2': {'r': 0.0, 'p': 0.0, 'f': 0.0}, 'rouge-l': {'r': 0.0, 'p': 0.0, 'f': 0.0}}, {'rouge-1': {'r': 0.06451612903225806, 'p': 0.2, 'f': 0.09756097192147545}, 'rouge-2': {'r': 0.0, 'p': 0.0, 'f': 0.0}, 'rouge-l': {'r': 0.06451612903225806, 'p': 0.2, 'f': 0.09756097192147545}}, {'rouge-1': {'r': 0.08333333333333333, 'p': 0.2222222222222222, 'f': 0.1212121172451792}, 'rouge-2': {'r': 0.038461538461538464, 'p': 0.1111111111111111, 'f': 0.05714285332244924}, 'rouge-l': {'r': 0.08333333333333333, 'p': 0.2222222222222222, 'f': 0.1212121172451792}}, {'rouge-1': {'r': 0.10526315789473684, 'p': 0.2222222222222222, 'f': 0.1428571384948981}, 'rouge-2': {'r': 0.0, 'p': 0.0, 'f': 0.0}, 'rouge-l': {'r': 0.10526315789473684, 'p': 0.2222222222222222, 'f': 0.1428571384948981}}, {'rouge-1': {'r': 0.1875, 'p': 0.2727272727272727, 'f': 0.22222221739369008}, 'rouge-2': {'r': 0.11764705882352941, 'p': 0.18181818181818182, 'f': 0.14285713808673